# Transformer - Gerador e Resumo


### Descriçao

---
1. O objetivo deste notebook é aprender sobre Transformer
2. Vamos aprender sobre a biblioteca Transformer e construir um gerador de texto e também um resumo de paragrafos.
---

### Baixando os pacotes


In [ ]:
!pip install transformers

### Documentação

1. **Transformers**:
   - [Documentação do Hugging Face Transformers](https://huggingface.co/transformers/)
   - [Repositório GitHub](https://github.com/huggingface/transformers)


2. **Datasets**:
   - [Documentação do Hugging Face Datasets](https://huggingface.co/docs/datasets/)
   - [Repositório GitHub](https://github.com/huggingface/datasets)


### Importando as bibliotecas

In [ ]:
import transformers
from transformers import pipeline #para realizar tarefas de PLN.


### Gerador de Texto

In [ ]:
gerador = pipeline("text-generation", model="pierreguillou/gpt2-small-portuguese") #o primeiro parametro é a tarefa! o segundo parametro é o
instrucao = """Me fale sobre Maycon Batestin."""

resultado = gerador(instrucao,
                    truncation=True,
                    max_length=600, #maximo de comprimento do texto
                    do_sample=True, #retorna uma amostragem estocagem, mais diversificada quando true
                    temperature=0.9, #vai de zero a 1 e quanto maior, mais criativa fica o texto. tmb define a tal alucinacao
                    top_k=50, # limita as proximas palavras ao numero k
                    top_p=0.85#limita o conjunto de palavras para um novo grupo
                   ) #ID do token de padding do GPT-2


In [ ]:
resultado = resultado[0]['generated_text']
resultado

In [ ]:
indice_fim_texto_original = resultado.find(instrucao)
novo_texto = resultado[indice_fim_texto_original + len(instrucao):].strip().replace('\n\n', ' ')
novo_texto

### Preencher Lacunas

In [ ]:
mascarar = pipeline("fill-mask", model="neuralmind/bert-base-portuguese-cased")# aqui trocamos a tarefa de gerar texto para preencher máscara. Preencher lacunas.

In [ ]:
prompt = mascarar("O Palmeiras conseguiu [MASK] o Mundial!") #onde ha o [] é onde vai haver o preenchimento
for x in range(len(prompt)):
  print(f"Probabilidade: {round(prompt[x]['score'],2) * 100}%")
  print(f"palavra sugerida: {prompt[x]['token_str'].upper()}")
  print(f"Frase preenchida: {prompt[x]['sequence']}")


### Perguntas e Respostas

In [ ]:
qea = pipeline("question-answering", model="pierreguillou/bert-base-cased-squad-v1.1-portuguese")

In [ ]:
context = """The Hitchhiker's Guide to the Galaxy (no Brasil O Guia do Mochileiro das Galáxias, em Portugal À Boleia Pela Galáxia)[1] é uma série de ficção científica cómica criada por Douglas Adams.[2] Originalmente era transmitida na BBC Radio 4 em 1978,[2] foi mais tarde adaptada para outros formatos, e após vários anos se tornou um fenômeno internacional. As adaptações incluíram espetáculos de palco, uma "trilogia" de cinco livros publicados entre 1979 e 1992,[1] um sexto romance escrito pelo Eoin Colfer, em 2009,[3] uma série de TV de 1981,[4] um jogo de computador de 1984,[5] e três séries de três partes de adaptações de romance para quadrinhos publicados pela DC Comics entre 1993 e 1996. Houve também duas toalhas, produzidas por Beer-Davies, que são considerados por alguns fãs como uma "versão oficial" do Guia do Mochileiro das Galáxias, uma vez que incluem o texto do primeiro romance.[6][7] Uma versão cinematográfica de Hollywood financiada, produzida e filmada no Reino Unido, foi lançada em abril de 2005, e adaptações de rádio dos terceiro, quarto e quinto romance foram transmitidos entre 2004 e 2005. Muitas dessas adaptações, incluindo os romances, as séries de TV, o jogo de computador, e os primeiros rascunhos do roteiro do filme de Hollywood, foram feitos pelo próprio Adams, e alguns da adaptação de palco mostra novo material escrito por Adams. """
prompt = "Quem escreveu o o livro?"
resposta = qea(question=prompt, context=context)
print("Pergunta: ", prompt)
print("Resposta: ", resposta['answer'])
print("Score: ", resposta['score'])

### Resumo

In [ ]:
resumidor = pipeline("summarization", model="Falconsai/text_summarization")


In [ ]:
prompt = """The Hitchhiker's Guide to the Galaxy (no Brasil O Guia do Mochileiro das Galáxias, em Portugal À Boleia Pela Galáxia)[1] é uma série de ficção científica cómica criada por Douglas Adams.[2] Originalmente era transmitida na BBC Radio 4 em 1978,[2] foi mais tarde adaptada para outros formatos, e após vários anos se tornou um fenômeno internacional. As adaptações incluíram espetáculos de palco, uma "trilogia" de cinco livros publicados entre 1979 e 1992,[1] um sexto romance escrito pelo Eoin Colfer, em 2009,[3] uma série de TV de 1981,[4] um jogo de computador de 1984,[5] e três séries de três partes de adaptações de romance para quadrinhos publicados pela DC Comics entre 1993 e 1996. Houve também duas toalhas, produzidas por Beer-Davies, que são considerados por alguns fãs como uma "versão oficial" do Guia do Mochileiro das Galáxias, uma vez que incluem o texto do primeiro romance.[6][7] Uma versão cinematográfica de Hollywood financiada, produzida e filmada no Reino Unido, foi lançada em abril de 2005, e adaptações de rádio dos terceiro, quarto e quinto romance foram transmitidos entre 2004 e 2005. Muitas dessas adaptações, incluindo os romances, as séries de TV, o jogo de computador, e os primeiros rascunhos do roteiro do filme de Hollywood, foram feitos pelo próprio Adams, e alguns da adaptação de palco mostra novo material escrito por Adams. """
resumo = resumidor(prompt, max_length=593, min_length=50)
resumo = resumo[0]['summary_text']
print(f"""Resumo:
{resumo}""")